In [1]:
import platform,os
os_name = platform.system()
hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]

In [2]:
print(hdfs_home)
print(os_name)

/user/kranthidr5051
Linux


In [3]:
path = hdfs_home+"/dataSets/spark-guide/activity-data/"

In [4]:
spark_home = "/usr/hdp/current/spark2-client"

In [5]:
import findspark
findspark.init(spark_home)
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [6]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("sg22-streamingAdvanced").getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("sg22-streamingAdvanced").getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://ip-172-31-35-141.ec2.internal:8088/proxy/application_1534772926501_6712


In [9]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [10]:
static = spark.read.json(path)

In [11]:
streaming = spark\
  .readStream\
  .schema(static.schema)\
  .option("maxFilesPerTrigger", 10)\
  .json(path)

In [12]:
static.show(10,False)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|Arrival_Time |Creation_Time      |Device  |Index|Model |User|gt   |x           |y           |z           |
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|18   |nexus4|g   |stand|3.356934E-4 |-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|66   |nexus4|g   |stand|-0.005722046|0.029083252 |0.005569458 |
|1424686735500|1424686733498505625|nexus4_1|99   |nexus4|g   |stand|0.0078125   |-0.017654419|0.010025024 |
|1424686735691|1424688581745026978|nexus4_2|145  |nexus4|g   |stand|-3.814697E-4|0.0184021   |-0.013656616|
|1424686735890|1424688581945252808|nexus4_2|185  |nexus4|g   |stand|-3.814697E-4|-0.031799316|-0.00831604 |
|1424686736094|1424686734097840342|nexus4_1|218  |nexus4|g   |stand|-7.324219E-4|-0.013381958|0.01109314  |
|1424686736294|1424688582347

In [13]:
static.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [14]:
static.selectExpr("cast(Creation_Time as double) as event_time").show(5,False)

+----------------------+
|event_time            |
+----------------------+
|1.42468673309063808E18|
|1.42468858134591821E18|
|1.42468673349850573E18|
|1.42468858174502707E18|
|1.42468858194525286E18|
+----------------------+
only showing top 5 rows



In [15]:
static.selectExpr("cast(Creation_Time as double)/1000000000 as event_time").show(5,False)

+--------------------+
|event_time          |
+--------------------+
|1.4246867330906382E9|
|1.4246885813459182E9|
|1.4246867334985058E9|
|1.424688581745027E9 |
|1.424688581945253E9 |
+--------------------+
only showing top 5 rows



In [16]:
static.selectExpr("cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time").show(5,False)

+--------------------------+
|event_time                |
+--------------------------+
|2015-02-23 10:18:53.090638|
|2015-02-23 10:49:41.345918|
|2015-02-23 10:18:53.498505|
|2015-02-23 10:49:41.745027|
|2015-02-23 10:49:41.945253|
+--------------------------+
only showing top 5 rows



In [17]:
# COMMAND ----------
withEventTime = streaming.selectExpr(
  "*",
  "cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")

In [18]:
# COMMAND ----------

from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes")).count()\
  .writeStream\
  .queryName("pyevents_per_window")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [19]:
from time import sleep
for x in range(3):
    spark.sql("SELECT * FROM pyevents_per_window").show(5, False)
    sleep(5)

+------+-----+
|window|count|
+------+-----+
+------+-----+

+------+-----+
|window|count|
+------+-----+
+------+-----+

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2015-02-24 11:50:00, 2015-02-24 12:00:00]|37714|
|[2015-02-24 13:00:00, 2015-02-24 13:10:00]|33324|
|[2015-02-23 12:30:00, 2015-02-23 12:40:00]|25218|
|[2015-02-23 10:20:00, 2015-02-23 10:30:00]|24856|
|[2015-02-24 12:30:00, 2015-02-24 12:40:00]|31496|
+------------------------------------------+-----+
only showing top 5 rows



In [20]:
spark.streams.active

In [21]:
# COMMAND ----------

from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
  .count()\
  .writeStream\
  .queryName("pyevents_per_window1")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [22]:
from time import sleep
for x in range(3):
    spark.sql("SELECT * FROM pyevents_per_window1").show(5, False)
    sleep(5)

+------+-----+
|window|count|
+------+-----+
+------+-----+

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2015-02-23 14:15:00, 2015-02-23 14:25:00]|13523|
|[2015-02-24 11:50:00, 2015-02-24 12:00:00]|18854|
|[2015-02-24 13:00:00, 2015-02-24 13:10:00]|16636|
|[2015-02-22 00:35:00, 2015-02-22 00:45:00]|5    |
|[2015-02-23 12:30:00, 2015-02-23 12:40:00]|12617|
+------------------------------------------+-----+
only showing top 5 rows

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2015-02-23 14:15:00, 2015-02-23 14:25:00]|26936|
|[2015-02-24 11:50:00, 2015-02-24 12:00:00]|37714|
|[2015-02-24 13:00:00, 2015-02-24 13:10:00]|33324|
|[2015-02-22 00:35:00, 2015-02-22 00:45:00]|6    |
|[2015-02-23 12:30:00, 2015-02-23 12:40:00]|25218|
+------------------------------------------+---

In [23]:
spark.streams.active

In [24]:
# COMMAND ----------

from pyspark.sql.functions import window, col
withEventTime\
  .withWatermark("event_time", "30 minutes")\
  .groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
  .count()\
  .writeStream\
  .queryName("pyevents_per_window2")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [ ]:
from time import sleep
for x in range(3):
    spark.sql("SELECT * FROM pyevents_per_window2").show(5, False)
    sleep(5)

+------+-----+
|window|count|
+------+-----+
+------+-----+



In [ ]:
spark.streams.active

In [ ]:
# COMMAND ----------

from pyspark.sql.functions import expr

withEventTime\
  .withWatermark("event_time", "5 seconds")\
  .dropDuplicates(["User", "event_time"])\
  .groupBy("User")\
  .count()\
  .writeStream\
  .queryName("pydeduplicated")\
  .format("memory")\
  .outputMode("complete")\
  .start()


# COMMAND ----------

In [ ]:
from time import sleep
for x in range(3):
    spark.sql("SELECT * FROM pydeduplicated").show(5, False)
    sleep(5)

In [ ]:
spark.streams.active